<a href="https://colab.research.google.com/github/Tara2712/Informatika/blob/main/Procesiranje_podatkov/ML_iskanje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.6 MB/s eta 0:00:00


In [2]:
df = pd.read_csv('df_no_nan_img.csv')
          # df = pd.read_csv('vzorec_podatkov.csv')


In [3]:
# df.head()

In [4]:
df.isna().sum()

,0
SR,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,0
DOLGI_OPIS_X,0


In [5]:
for col in ["NAZIV_SR", "OPIS", "DOLGI_OPIS_X"]:
  df[col] = df[col].fillna("")

In [6]:
df["text"] = df["NAZIV_SR"] + " " + df["OPIS"] + " " + df["DOLGI_OPIS_X"]

In [7]:
# df.head()

že natreniran model za transformacijo besed, ki išče semantično podobnost med povedmi

In [8]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

uporaba modela:
-  v novem stolpcu shrani numeričen embedding za vsak text stolpec
- na tak način lahko izračuna pomenske podobnosti med besedili


In [9]:
df["embedding"] = df["text"].apply(lambda x: model.encode(x))

In [10]:
def search(query, min_similarity=0.4):
  query_embedding = model.encode(query).reshape(1, -1) # input spremeni v embeding (isto kot so vse vrednosti v df)
  all_embeddings = np.stack(df["embedding"].values) # vse embeddinge ki so bile narejene za text stacka v NumPy array

  similarities = cosine_similarity(query_embedding, all_embeddings)[0] # izračuna kosinusno podobnost med vnosnim besedilom in vsemi ostalimi v df-u
  top_indices = similarities.argsort()[::-1] # razvrsti vse vnose v df-u glede na podobnost, od največjega do najmanjšega

  filtered_indices = [i for i in top_indices if similarities[i] >= min_similarity] # sfiltrira top_indices tako, da vključi sam tiste ki imajo podobnost večjo od zahtevane

  results = df.iloc[filtered_indices][["SR", "NAZIV_SR", "OPIS", "DOLGI_OPIS_X"]].copy() # izbere tiste vrstice iz df, ki spadajo pod filtered_indices
  results["similarity"] = [similarities[i] for i in filtered_indices] # zapiše kakšna je podobnost
  return results

In [11]:
search("harry potter", min_similarity=0.5)

,SR,NAZIV_SR,OPIS,DOLGI_OPIS_X,similarity


In [12]:
df.columns

Index(['SR', 'NAZIV_SR', 'DATUM_NASTANKA_SR', 'STATUS', 'WORKLOGID', 'OPIS',
       'DOLGI_OPIS_X', 'text', 'embedding'],
      dtype='object')